<link href="https://fonts.googleapis.com/css2?family=Inter:wght@400&display=swap" rel="stylesheet">
<div style="font-family: 'Inter'; font-size: 24px; color: #749857;"><B>🍏 Show Up</b> for Health</div>
<div style="font-family: 'Inter'; font-size: 14px; color: #7a7979;"><B>Predicting Missed Apppointment in Primary Care</b> - Brompton Health PCN</div>

[janduplessis883](https://github.com/janduplessis883)<BR>
[AlexAlexRose](https://github.com/AlexAlexRose)<BR>
[FabySp](https://github.com/FabySp)<BR>
[mogleyza](https://github.com/mogleyza)

# GitHub Username | Notebook Title 

### Importing Libraries

In [16]:
# Importing default Libraries
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import seaborn as sns
import warnings

# Hi-resolution Plots and Matplotlib inline
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# Set the maximum number of rows and columns to be displayed
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
warnings.filterwarnings('ignore')

# "magic commands" to enable autoreload of your imported packages
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Params

In [7]:
RAW_DATA = '~/code/janduplessis883/data-showup/data/raw-data/'
WEATHER_DATA = '~/code/janduplessis883/data-showup/data/weather/weather.csv'
IMD_DATA = '~/code/janduplessis883/data-showup/data/imd-master/imd_master.csv'

OUTPUT_DATA = '~/code/janduplessis883/data-showup/data/output-data/'

In [8]:
import math

def haversine_distance(surgery_prefix, lat2, lon2):
    R = 6371.0  # Radius of the Earth in kilometers

    if surgery_prefix == 'ECS':
        lat1, lon1 = 51.488721, -0.191873
    elif surgery_prefix == 'SMW':
        lat1, lon1 = 51.494474, -0.181931
    elif surgery_prefix == 'TCP':
        lat1, lon1 = 51.48459, -0.171887
    elif surgery_prefix == 'HPVM':
        lat1, lon1 = 51.48459, -0.171887
    elif surgery_prefix == 'KMC':
        lat1, lon1 = 51.49807, -0.159918
    elif surgery_prefix == 'TGP':
        lat1, lon1 = 51.482652, -0.178066


    # Convert degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Differences
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance  # in kilometers

In [9]:
def make_global_disease_register(surgery_list = ['ECS', 'TCP', 'TGP', 'SMW', 'KMC', 'HPVM']):
    print('=== Preparing Global Disease Register + IMD2023 info ======================================')

    disease_register = []
    for surgery in surgery_list:
        register_path = f'{RAW_DATA}{surgery}/{surgery}'

        idnhs = pd.read_excel(f'{register_path}_NHS_PTID.xlsx', dtype='str')
        idnhs.dropna(inplace=True)
        frail = pd.read_csv(f'{register_path}_FRAILTY.csv', dtype='str')
        dep = pd.read_csv(f'{register_path}_DEPRESSION.csv', dtype='str')
        obesity = pd.read_csv(f'{register_path}_OBESITY.csv', dtype='str')
        chd = pd.read_csv(f'{register_path}_IHD.csv', dtype='str')
        dm = pd.read_csv(f'{register_path}_DM.csv', dtype='str')
        hpt = pd.read_csv(f'{register_path}_HPT.csv', dtype='str')
        ndhg = pd.read_csv(f'{register_path}_NDHG.csv', dtype='str')
        smi = pd.read_csv(f'{register_path}_SMI.csv', dtype='str')

        ptid = idnhs.merge(frail, how='left', on='NHS number')
        ptid = ptid.drop(columns='NHS number')

        register = (ptid.merge(dep, how='left', on='Patient ID')
                    .merge(obesity, how='left', on='Patient ID')
                    .merge(chd, how='left', on='Patient ID')
                    .merge(dm, how='left', on='Patient ID')
                    .merge(hpt, how='left', on='Patient ID')
                    .merge(ndhg, how='left', on='Patient ID')
                    .merge(smi, how='left', on='Patient ID')
                    .fillna(0)
                    )
        print(f'💊 {surgery} Disease Register completed')
        # Add IMD and distance from station
        imd = pd.read_csv(IMD_DATA)

        full_register = register.merge(imd, how='left', on='Postcode')
        print(f'🔸 {surgery} IMD2023')
        full_register['distance_from_surg'] = full_register.apply(lambda row: haversine_distance(surgery, row['Latitude'], row['Longitude']), axis=1)
        disease_register.append(full_register)

    global_register = pd.concat(disease_register, axis=0, ignore_index=True)
    print(f"🦠 Concat Registers into ONE REGISTER")
    
    global_register.dropna(inplace=True)
    print(f'❌ Dropped NaN')
    
    output_path = f'{OUTPUT_DATA}global_disease_register.csv'
    global_register.to_csv(output_path, index=False)
    print(f'✅ Global Disease Register Saved to output-data: {global_register.shape}')
    print()
    return global_register

In [10]:
disease_register = make_global_disease_register()

=== Preparing Global Disease Register + IMD2023 info ======================================
💊 ECS Disease Register completed
🔸 ECS IMD2023
💊 TCP Disease Register completed
🔸 TCP IMD2023
💊 TGP Disease Register completed
🔸 TGP IMD2023
💊 SMW Disease Register completed
🔸 SMW IMD2023
💊 KMC Disease Register completed
🔸 KMC IMD2023
💊 HPVM Disease Register completed
🔸 HPVM IMD2023
🦠 Concat Registers into ONE REGISTER
❌ Dropped NaN
✅ Global Disease Register Saved to output-data: (7344, 21)



In [17]:
data = pd.read_csv(f'{RAW_DATA}ECS/ECS_CLINICAL.csv')
data

,NHS number,WEIGHT,HEIGHT,Depression,SMI,BP,HBA1C,DM,IHD,TYPE REG
0,651 240 3436,55 Kg,1.55 m,,,122 / 66,43 mmol/mol,,,GMS
1,651 240 3436,55 Kg,1.55 m,,,122 / 66,43 mmol/mol,,,GMS
2,626 457 1857,67 Kg,1.7 m,,,122 / 66,50 mmol/mol,X40J5,,GMS
3,703 981 2533,87 Kg,1.89 m,,,134 / 86,NaN,,,GMS
4,628 208 6345,NaN,NaN,,,NaN,NaN,,,GMS
...,...,...,...,...,...,...,...,...,...,...
9856,637 452 2559,57 Kg,1.68 m,,,117 / 81,38 mmol/mol,,,GMS
9857,640 136 6478,56 Kg,1.7 m,,,102 / 66,34 mmol/mol,,,GMS
9858,624 957 3909,70 Kg,1.72 m,,,108 / 56,NaN,,,GMS
9859,624 957 3909,70 Kg,1.72 m,,,108 / 56,NaN,,,GMS


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9861 entries, 0 to 9860
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   NHS number  9667 non-null   object
 1   WEIGHT      8698 non-null   object
 2   HEIGHT      8409 non-null   object
 3   Depression  9861 non-null   object
 4   SMI         9861 non-null   object
 5   BP          7645 non-null   object
 6   HBA1C       3825 non-null   object
 7   DM          9861 non-null   object
 8   IHD         9861 non-null   object
 9   TYPE REG    9861 non-null   object
dtypes: object(10)
memory usage: 770.5+ KB


In [21]:
# Split the 'BP' column into two separate columns
data[['SYS', 'DIAS']] = data['BP'].str.split('/', expand=True)

# Convert 'SYS' and 'DIAS' columns to numeric (integer) type
data['SYS'] = pd.to_numeric(data['SYS'], errors='coerce')
data['DIAS'] = pd.to_numeric(data['DIAS'], errors='coerce')

# Display the updated DataFrame
data

,NHS number,WEIGHT,HEIGHT,Depression,SMI,BP,HBA1C,DM,IHD,TYPE REG,SYS,DIAS
0,651 240 3436,55 Kg,1.55 m,,,122 / 66,43 mmol/mol,,,GMS,122.0,66.0
1,651 240 3436,55 Kg,1.55 m,,,122 / 66,43 mmol/mol,,,GMS,122.0,66.0
2,626 457 1857,67 Kg,1.7 m,,,122 / 66,50 mmol/mol,X40J5,,GMS,122.0,66.0
3,703 981 2533,87 Kg,1.89 m,,,134 / 86,NaN,,,GMS,134.0,86.0
4,628 208 6345,NaN,NaN,,,NaN,NaN,,,GMS,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9856,637 452 2559,57 Kg,1.68 m,,,117 / 81,38 mmol/mol,,,GMS,117.0,81.0
9857,640 136 6478,56 Kg,1.7 m,,,102 / 66,34 mmol/mol,,,GMS,102.0,66.0
9858,624 957 3909,70 Kg,1.72 m,,,108 / 56,NaN,,,GMS,108.0,56.0
9859,624 957 3909,70 Kg,1.72 m,,,108 / 56,NaN,,,GMS,108.0,56.0


In [24]:
data['Numeric_HBa1c'] = data['HBA1C'].str.extract(r'(\d+)').astype(float)
data

,NHS number,WEIGHT,HEIGHT,Depression,SMI,BP,HBA1C,DM,IHD,TYPE REG,SYS,DIAS,Numeric_HBa1c
0,651 240 3436,55 Kg,1.55 m,,,122 / 66,43 mmol/mol,,,GMS,122.0,66.0,43.0
1,651 240 3436,55 Kg,1.55 m,,,122 / 66,43 mmol/mol,,,GMS,122.0,66.0,43.0
2,626 457 1857,67 Kg,1.7 m,,,122 / 66,50 mmol/mol,X40J5,,GMS,122.0,66.0,50.0
3,703 981 2533,87 Kg,1.89 m,,,134 / 86,NaN,,,GMS,134.0,86.0,NaN
4,628 208 6345,NaN,NaN,,,NaN,NaN,,,GMS,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9856,637 452 2559,57 Kg,1.68 m,,,117 / 81,38 mmol/mol,,,GMS,117.0,81.0,38.0
9857,640 136 6478,56 Kg,1.7 m,,,102 / 66,34 mmol/mol,,,GMS,102.0,66.0,34.0
9858,624 957 3909,70 Kg,1.72 m,,,108 / 56,NaN,,,GMS,108.0,56.0,NaN
9859,624 957 3909,70 Kg,1.72 m,,,108 / 56,NaN,,,GMS,108.0,56.0,NaN


In [25]:
# Define a function to apply the condition
def check_NDHG(value):
    if value > 42 and value < 48:
        return 1
    else:
        return 0

# Use the apply function to create a new 'NDHG' column
data['NDHG'] = data['Numeric_HBa1c'].apply(check_NDHG)
data

,NHS number,WEIGHT,HEIGHT,Depression,SMI,BP,HBA1C,DM,IHD,TYPE REG,SYS,DIAS,Numeric_HBa1c,NDHG
0,651 240 3436,55 Kg,1.55 m,,,122 / 66,43 mmol/mol,,,GMS,122.0,66.0,43.0,1
1,651 240 3436,55 Kg,1.55 m,,,122 / 66,43 mmol/mol,,,GMS,122.0,66.0,43.0,1
2,626 457 1857,67 Kg,1.7 m,,,122 / 66,50 mmol/mol,X40J5,,GMS,122.0,66.0,50.0,0
3,703 981 2533,87 Kg,1.89 m,,,134 / 86,NaN,,,GMS,134.0,86.0,NaN,0
4,628 208 6345,NaN,NaN,,,NaN,NaN,,,GMS,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9856,637 452 2559,57 Kg,1.68 m,,,117 / 81,38 mmol/mol,,,GMS,117.0,81.0,38.0,0
9857,640 136 6478,56 Kg,1.7 m,,,102 / 66,34 mmol/mol,,,GMS,102.0,66.0,34.0,0
9858,624 957 3909,70 Kg,1.72 m,,,108 / 56,NaN,,,GMS,108.0,56.0,NaN,0
9859,624 957 3909,70 Kg,1.72 m,,,108 / 56,NaN,,,GMS,108.0,56.0,NaN,0


In [ ]:
# Define a function to apply the condition
def check_DM2(row):
    if pd.notna(row['DM']) or row['Numeric_HBa1c'] >= 48:
        return 1
    else:
        return 0

# Use the apply function to create a new 'DM2' column
df['DM2'] = df.apply(check_DM2, axis=1)

# Display the updated DataFrame
print(df)